In [1]:
import numpy as np
import tensorflow as tf


print('{} version {}'.format(np.__name__, np.__version__))
print('{} version {}'.format(tf.__name__, tf.__version__))
print("Num GPU Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print('GPU list: ', tf.config.list_physical_devices('GPU'))

a = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
], dtype=np.int) # 4 bytes
print('Memory block address {}'.format(a.__array_interface__['data'][0]))
print('2D Shape: {} 2D Strides: {} Size {}'.format(a.shape, a.strides, a.size))

a3D = np.array([
    [
        [0.00, 0.10, 0.20, 0.30],
        [1.00, 1.10, 1.20, 1.30],
        [2.00, 2.10, 2.20, 2.30]
    ],
    [
        [0.01, 0.11, 0.21, 0.31],
        [1.01, 1.11, 1.21, 1.31],
        [2.01, 2.11, 2.21, 2.31]
    ],
    [
        [0.01, 0.12, 0.22, 0.32],
        [1.02, 1.12, 1.22, 1.32],
        [2.02, 2.12, 2.22, 2.32]
    ]
], dtype=np.float) # = 8 bytes
print('3D Shape: {} 3D Strides: {}'.format(a3D.shape, a3D.strides))
a3D[2,:,:] # last slice


numpy version 1.18.5
tensorflow version 2.4.1
Num GPU Available:  0
GPU list:  []
Memory block address 2706668407152
2D Shape: (3, 4) 2D Strides: (16, 4) Size 12
3D Shape: (3, 3, 4) 3D Strides: (96, 32, 8)


array([[0.01, 0.12, 0.22, 0.32],
       [1.02, 1.12, 1.22, 1.32],
       [2.02, 2.12, 2.22, 2.32]])

# Basis reshape operations

Reshape is immutable

New 'view' to the array is returned and the origin memory is shared.

Only shapes ans strides are changes

In [315]:

r = a.reshape(2, 6)
assert np.shares_memory(a, r)
print('Shape: {} Strides: {}'.format(r.shape, r.strides))

Shape: (2, 6) Strides: (24, 4)


In [316]:
r = a.reshape(-1, 1) # -1 means 'guess by yourself'
print('Shape: {} Strides: {}'.format(r.shape, r.strides))

Shape: (12, 1) Strides: (4, 4)


### Practcal example. Sliding window with means

In [325]:
plain = np.array([1,2,3,4,5,6,7,8,9])
r = plain.reshape(3, 3)
mean = r.mean(axis=1) # row means because axis 1 is columns
sum = r.sum(axis=1) # row sum
print('Means: {} Sum {}'.format(mean, sum))

Means: [2. 5. 8.] Sum [ 6 15 24]


### Stride Tricks

In [317]:
r = np.lib.stride_tricks.as_strided(a, (2, 6), (24, 4)) # The same as a.reshape(2, 6)
assert np.shares_memory(a, r)
r = np.lib.stride_tricks.as_strided(a, (30, 40), (0, 4))
# Now Numpy believes that this array contains 30x40 different elements, whereas the data buffer actually 
# contains the same 3x4 elements as original array a
assert np.shares_memory(a, r)
r

array([[1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       ...,
       [1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0],
       [1, 2, 3, ..., 0, 0, 0]])

In [318]:
a3D.reshape(4, 3, 3)

array([[[0.  , 0.1 , 0.2 ],
        [0.3 , 1.  , 1.1 ],
        [1.2 , 1.3 , 2.  ]],

       [[2.1 , 2.2 , 2.3 ],
        [0.01, 0.11, 0.21],
        [0.31, 1.01, 1.11]],

       [[1.21, 1.31, 2.01],
        [2.11, 2.21, 2.31],
        [0.01, 0.12, 0.22]],

       [[0.32, 1.02, 1.12],
        [1.22, 1.32, 2.02],
        [2.12, 2.22, 2.32]]])

# Transpose

Transpose is also immutable.

It also returns new object with different properties, but the array's memory is shared

Transpose is actually reverses the shape and strides

but (more importantly) changes the row-major (C-type) to column-major (Fortran-type)

In [4]:
# %timeit 
b = a.T  # .T is a property of np.array object
assert np.shares_memory(a, b)
print('Transposed Shapes: {} Strides: {}'.format(b.shape, b.strides ))
assert b.data.f_contiguous
b[0, 0] = 100 # a[0, 0] is changes accordingly


Transposed Shapes: (4, 3) Strides: (4, 16)


In [ ]:
# %%timeit
aa = np.random.rand(12, 64)
print('Shapes: {} Strides {}'.format(aa.shape, aa.strides))
bb = aa.T # get_ipython().run_line_magic('timeit', 'bb = aa.T')
print('Transposed Shapes: {} Strides {}'.format(bb.shape, bb.strides))
assert np.shares_memory(aa, bb)

# Tensorflow

In [322]:
tf.debugging.set_log_device_placement(True)
# with tf.device('/CPU:0'):
tensor = tf.constant(a)
tf.print(tensor, output_stream=sys.stdout, sep=',')

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])
model.compile(loss='mean_squared_error',
    metrics=['accuracy'],
    optimizer=tf.keras.optimizers.Adam(0.1) )
model.summary()

[[1 2 3 4]
 [5 6 7 8]
 [9 10 11 12]]
Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
